In [18]:
import pandas as pd
from data import citylearn_challenge_2022_phase_1 as competition_data
import os.path as osp
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from matplotlib import pyplot as plt
from skforecast.model_selection import backtesting_forecaster

consumptions_path = osp.join(osp.dirname(competition_data.__file__), "consumptions/building_consumptions.csv")
consumptions = pd.read_csv(consumptions_path)[[f"{i}" for i in range(5)]]
# consumptions

In [19]:
consumption_building_1 = consumptions["0"].to_frame().rename(columns={"0":"Consumption"})
consumption_building_2 = consumptions["1"].to_frame().rename(columns={"1":"Consumption"})
consumption_building_3 = consumptions["2"].to_frame().rename(columns={"2":"Consumption"})
consumption_building_4 = consumptions["3"].to_frame().rename(columns={"3":"Consumption"})
consumption_building_5 = consumptions["4"].to_frame().rename(columns={"4":"Consumption"})

In [20]:
data = consumption_building_1

In [21]:
# Lags used as predictors
load_lags_grid = [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41],
                  [1],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                  [1, 23, 45, 67],
                  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
                  [1, 23, 45],
                  [1, 2, 3, 4, 5],
                  [1, 2, 3, 4, 5, 6],
                  [1, 2, 3, 4, 23, 24, 25, 26, 45, 46, 47, 48, 67, 68, 69, 70]]


In [22]:
solar_lags_grid = [
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
[1, 2, 23, 24, 45, 46],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
[1],
[1, 2, 3, 4, 5, 6, 23, 24, 25, 26, 27, 28],
[1, 2],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63],
[1, 2, 23, 24, 45, 46, 67, 68],
[1, 2, 23, 24, 45, 46],
]

In [27]:
def run_grid_search(ys, lags_grid):
    all_results = pd.DataFrame()
    for y in ys:

        forecaster = ForecasterAutoreg(
                     regressor     = Ridge(random_state=123),
                     lags          = 24, # This value will be replaced in the grid search
                     transformer_y = StandardScaler()
                 )

        forecaster.fit(y=y)

        # Regressor's hyperparameters
        param_grid = {'alpha': np.logspace(-3, 5, 10),
                      'tol': np.logspace(-1, -10, 10),
                      'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}


        results_grid = grid_search_forecaster(
                           forecaster         = forecaster,
                           y                  = y,
                           param_grid         = param_grid, # param_grid,
                           lags_grid          = lags_grid,
                           steps              = 1,
                           metric             = 'mean_squared_error',
                           refit              = False,
                           initial_train_size = len(y) - 1,
                           fixed_train_size   = False,
                           return_best        = True,
                           verbose            = False
                       )

        all_results = all_results.append(results_grid[:3])

    return all_results



In [28]:
# results = run_grid_search([consumption_building_1.Consumption, consumption_building_2.Consumption, consumption_building_3.Consumption, consumption_building_4.Consumption, consumption_building_5.Consumption])

In [29]:
nsload_data_path = osp.join(osp.dirname(competition_data.__file__), "load_data.csv")
df_loads = pd.read_csv(nsload_data_path)
df_load_1 = df_loads[:8760]["non_shiftable_load"].to_frame()
df_load_2 = df_loads[8760:8760*2]["non_shiftable_load"].to_frame()
df_load_3 = df_loads[8760*2:8760*3]["non_shiftable_load"].to_frame()
df_load_4 = df_loads[8760*3:8760*4]["non_shiftable_load"].to_frame()
df_load_5 = df_loads[8760*4:8760*5]["non_shiftable_load"].to_frame()

In [ ]:
run_grid_search([df_load_1.non_shiftable_load, df_load_2.non_shiftable_load, df_load_3.non_shiftable_load, df_load_4.non_shiftable_load, df_load_5.non_shiftable_load], load_lags_grid)

Number of models compared: 10500.


loop lags_grid: 100%|█████████████████████████████████████| 15/15 [12:38<00:00, 50.57s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 23 24 25 26 27 28 29 30
 31 32 33 34 35 36 37 38 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60] 
  Parameters: {'alpha': 3.593813663804626, 'solver': 'sag', 'tol': 0.001}
  Backtesting metric: 1.6570807747339501e-06

Number of models compared: 10500.


loop lags_grid: 100%|█████████████████████████████████████| 15/15 [12:28<00:00, 49.88s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 23 24 25 26 27 28 29 30 31
 32 33 34 35 36 37 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59] 
  Parameters: {'alpha': 0.001, 'solver': 'sag', 'tol': 0.1}
  Backtesting metric: 0.003901924484038252

Number of models compared: 10500.


loop lags_grid: 100%|█████████████████████████████████████| 15/15 [12:27<00:00, 49.82s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1 23 45] 
  Parameters: {'alpha': 12915.496650148827, 'solver': 'sparse_cg', 'tol': 0.1}
  Backtesting metric: 5.158026788299217e-07

Number of models compared: 10500.


loop param_grid:  68%|███████████████████████           | 476/700 [00:42<00:21, 10.64it/s]

In [16]:
nsload_data_path = osp.join(osp.dirname(competition_data.__file__), "solar_data.csv")
df_solars = pd.read_csv(nsload_data_path)
df_solar_1 = df_solars[:8760]["solar_generation"].to_frame()
df_solar_2 = df_solars[8760:8760*2]["solar_generation"].to_frame()
df_solar_3 = df_solars[8760*2:8760*3]["solar_generation"].to_frame()
df_solar_4 = df_solars[8760*3:8760*4]["solar_generation"].to_frame()
df_solar_5 = df_solars[8760*4:8760*5]["solar_generation"].to_frame()

In [17]:
run_grid_search([df_solar_1.solar_generation, df_solar_2.solar_generation, df_solar_3.solar_generation, df_solar_4.solar_generation, df_solar_5.solar_generation], solar_lags_grid)

Number of models compared: 63.


loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 45 46 47 48 49 50 51 52 53 54
 55 56 57 58 59 60 61 62 63] 
  Parameters: {}
  Backtesting metric: 2.560242194106592e-11

Number of models compared: 63.


loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 23 24 25 26 27 28 29 30
 31 32 33 34 35 36 37 38 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60] 
  Parameters: {}
  Backtesting metric: 4.983548675825438e-10

Number of models compared: 63.


...

loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2] 
  Parameters: {}
  Backtesting metric: 0.07190111353866258

Number of models compared: 63.


loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]
                                                                                          
loop param_grid:   0%|                                              | 0/1 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 45 46 47 48 49 50 51 52 53 54
 55 56 57 58 59 60 61 62 63] 
  Parameters: {}
  Backtesting metric: 1.3725946512825515e-09



,lags,params,mean_squared_error
56,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},2.560242e-11
36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},6.669885e-07
53,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},1.117649e-06
53,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},4.983549e-10
39,"[1, 2, 23, 24, 45, 46]",{},2.785274e-08
35,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{},3.315055e-08
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",{},2.685317e-05
0,[1],{},1.539516e-04
24,"[1, 2, 3, 4, 5, 6, 23, 24, 25, 26, 27, 28]",{},3.436608e-04
1,"[1, 2]",{},7.190111e-02
